In [9]:
# Notebook originally by Randal S. Olson, modified by Vinicius Canto Costa to change the data used.
# The waypoints are in Brazil

all_waypoints = ["Sugarloaf Mountain Pão de Açúcar, RJ, Brazil",\
                        "Copacabana Palace, A Belmond Hotel, RJ, Brazil", \
                        "Liberdade São Paulo - SP, Brazil", \
                        "Campos do Jordão SP, Brazil", \
                        "Itaúnas Conceição da Barra - ES, Brazil", \
                        "Elevador Lacerda BA, Brazil", \
                        "Porto Seguro BA, Brazil",\
                        "Fundação Projeto Tamar - Aracaju, Av. Santos Dumont, 1010 - Atalaia, Aracaju - SE, Brazil",\
                        "Maragogi AL, Brazil",\
                        "Olinda PE, Brazil",\
                        "Petrolina PE, Brazil", \
                        "Ponta do Seixas João Pessoa - PB, Brazil",\
                        "Genipabu Praia de Genipabu, RN, Brazil", \
                        "Cashew of Pirangi, Pirangi do Norte, Parnamirim - RN, Brazil",\
                        "Fortaleza, CE, Brazil", \
                        "Jericoacoara, CE, Brazil", \
                        "Parque Nacional Serra da Capivara, PI, Brazil",\
                        "Parque Nacional dos Lençóis Maranhenses, MA, Brazil",\
                        "Fordlândia Aveiro PA, Brazil",\
                        "Belem, PA, Brazil",\
                         "Manaus, AM, Brazil", \
                         "Boa Vista, RR, Brazil", \
                         "Rio Branco, AC, Brazil", \
                         "Porto Velho, RO, Brazil", \
                         "Cuiaba, MT, Brazil",\
                         "Bonito, MS, Brazil",\
                         "Caldas Novas, GO, Brazil",\
                         "Praça dos Três Poderes, DF, Brazil",\
                         "Palmas, TO, Brazil",\
                         "Ouro Preto, MG, Brazil",\
                         "Iguazu Falls, PR, Brazil",\
                         "Balneario Camboriu, SC, Brazil",\
                         "Blumenau, SC, Brazil",\
                         "Gramado, RS, Brazil"]
len(all_waypoints)

34

In [6]:
import googlemaps

gmaps = googlemaps.Client(key="PASTE YOUR API KEY HERE")

In [10]:
from itertools import combinations

waypoint_distances = {}
waypoint_durations = {}

for (waypoint1, waypoint2) in combinations(all_waypoints, 2):
    try:
        route = gmaps.distance_matrix(origins=[waypoint1],
                                      destinations=[waypoint2],
                                      mode="driving", # Change this to "walking" for walking directions,
                                                      # "bicycling" for biking directions, etc.
                                      language="English",
                                      units="metric")

        # "distance" is in meters
        distance = route["rows"][0]["elements"][0]["distance"]["value"]

        # "duration" is in seconds
        duration = route["rows"][0]["elements"][0]["duration"]["value"]

        waypoint_distances[frozenset([waypoint1, waypoint2])] = distance
        waypoint_durations[frozenset([waypoint1, waypoint2])] = duration
    
    except Exception as e:
        print("Error with finding the route between %s and %s." % (waypoint1, waypoint2))

In [11]:
with open("my-waypoints-dist-dur_br.tsv", "w") as out_file:
    out_file.write("\t".join(["waypoint1",
                              "waypoint2",
                              "distance_m",
                              "duration_s"]))
    
    for (waypoint1, waypoint2) in waypoint_distances.keys():
        out_file.write("\n" +
                       "\t".join([waypoint1,
                                  waypoint2,
                                  str(waypoint_distances[frozenset([waypoint1, waypoint2])]),
                                  str(waypoint_durations[frozenset([waypoint1, waypoint2])])]))

In [12]:
import pandas as pd
import numpy as np

waypoint_distances = {}
waypoint_durations = {}
all_waypoints = set()

waypoint_data = pd.read_csv("my-waypoints-dist-dur_br.tsv", sep="\t")

for i, row in waypoint_data.iterrows():
    waypoint_distances[frozenset([row.waypoint1, row.waypoint2])] = row.distance_m
    waypoint_durations[frozenset([row.waypoint1, row.waypoint2])] = row.duration_s
    all_waypoints.update([row.waypoint1, row.waypoint2])

In [13]:
import random

def compute_fitness(solution):
    """
        This function returns the total distance traveled on the current road trip.
        
        The genetic algorithm will favor road trips that have shorter
        total distances traveled.
    """
    
    solution_fitness = 0.0
    
    for index in range(len(solution)):
        waypoint1 = solution[index - 1]
        waypoint2 = solution[index]
        solution_fitness += waypoint_distances[frozenset([waypoint1, waypoint2])]
        
    return solution_fitness

def generate_random_agent():
    """
        Creates a random road trip from the waypoints.
    """
    
    new_random_agent = list(all_waypoints)
    random.shuffle(new_random_agent)
    return tuple(new_random_agent)

def mutate_agent(agent_genome, max_mutations=3):
    """
        Applies 1 - `max_mutations` point mutations to the given road trip.
        
        A point mutation swaps the order of two waypoints in the road trip.
    """
    
    agent_genome = list(agent_genome)
    num_mutations = random.randint(1, max_mutations)
    
    for mutation in range(num_mutations):
        swap_index1 = random.randint(0, len(agent_genome) - 1)
        swap_index2 = swap_index1

        while swap_index1 == swap_index2:
            swap_index2 = random.randint(0, len(agent_genome) - 1)

        agent_genome[swap_index1], agent_genome[swap_index2] = agent_genome[swap_index2], agent_genome[swap_index1]
            
    return tuple(agent_genome)

def shuffle_mutation(agent_genome):
    """
        Applies a single shuffle mutation to the given road trip.
        
        A shuffle mutation takes a random sub-section of the road trip
        and moves it to another location in the road trip.
    """
    
    agent_genome = list(agent_genome)
    
    start_index = random.randint(0, len(agent_genome) - 1)
    length = random.randint(2, 20)
    
    genome_subset = agent_genome[start_index:start_index + length]
    agent_genome = agent_genome[:start_index] + agent_genome[start_index + length:]
    
    insert_index = random.randint(0, len(agent_genome) + len(genome_subset) - 1)
    agent_genome = agent_genome[:insert_index] + genome_subset + agent_genome[insert_index:]
    
    return tuple(agent_genome)

def generate_random_population(pop_size):
    """
        Generates a list with `pop_size` number of random road trips.
    """
    
    random_population = []
    for agent in range(pop_size):
        random_population.append(generate_random_agent())
    return random_population
    
def run_genetic_algorithm(generations=5000, population_size=100):
    """
        The core of the Genetic Algorithm.
        
        `generations` and `population_size` must be a multiple of 10.
    """
    
    population_subset_size = int(population_size / 10.)
    generations_10pct = int(generations / 10.)
    
    # Create a random population of `population_size` number of solutions.
    population = generate_random_population(population_size)

    # For `generations` number of repetitions...
    for generation in range(generations):
        
        # Compute the fitness of the entire current population
        population_fitness = {}

        for agent_genome in population:
            if agent_genome in population_fitness:
                continue

            population_fitness[agent_genome] = compute_fitness(agent_genome)

        # Take the top 10% shortest road trips and produce offspring each from them
        new_population = []
        for rank, agent_genome in enumerate(sorted(population_fitness,
                                                   key=population_fitness.get)[:population_subset_size]):
            
            if (generation % generations_10pct == 0 or generation == generations - 1) and rank == 0:
                print("Generation %d best: %d | Unique genomes: %d" % (generation,
                                                                       population_fitness[agent_genome],
                                                                       len(population_fitness)))
                print(agent_genome)
                print("")

            # Create 1 exact copy of each of the top road trips
            new_population.append(agent_genome)

            # Create 2 offspring with 1-3 point mutations
            for offspring in range(2):
                new_population.append(mutate_agent(agent_genome, 3))
                
            # Create 7 offspring with a single shuffle mutation
            for offspring in range(7):
                new_population.append(shuffle_mutation(agent_genome))

        # Replace the old population with the new population of offspring 
        for i in range(len(population))[::-1]:
            del population[i]

        population = new_population

In [14]:
run_genetic_algorithm(generations=5000, population_size=100)

Generation 0 best: 69055667 | Unique genomes: 100
('Copacabana Palace, A Belmond Hotel, RJ, Brazil', 'Bonito, MS, Brazil', 'Iguazu Falls, PR, Brazil', 'Boa Vista, RR, Brazil', 'Blumenau, SC, Brazil', 'Manaus, AM, Brazil', 'Porto Velho, RO, Brazil', 'Fundação Projeto Tamar - Aracaju, Av. Santos Dumont, 1010 - Atalaia, Aracaju - SE, Brazil', 'Olinda PE, Brazil', 'Petrolina PE, Brazil', 'Praça dos Três Poderes, DF, Brazil', 'Rio Branco, AC, Brazil', 'Genipabu Praia de Genipabu, RN, Brazil', 'Ponta do Seixas João Pessoa - PB, Brazil', 'Ouro Preto, MG, Brazil', 'Parque Nacional Serra da Capivara, PI, Brazil', 'Jericoacoara, CE, Brazil', 'Sugarloaf Mountain Pão de Açúcar, RJ, Brazil', 'Maragogi AL, Brazil', 'Parque Nacional dos Lençóis Maranhenses, MA, Brazil', 'Porto Seguro BA, Brazil', 'Liberdade São Paulo - SP, Brazil', 'Balneario Camboriu, SC, Brazil', 'Cuiaba, MT, Brazil', 'Palmas, TO, Brazil', 'Campos do Jordão SP, Brazil', 'Gramado, RS, Brazil', 'Caldas Novas, GO, Brazil', 'Fortaleza,

Generation 3500 best: 23526064 | Unique genomes: 91
('Elevador Lacerda BA, Brazil', 'Porto Seguro BA, Brazil', 'Itaúnas Conceição da Barra - ES, Brazil', 'Ouro Preto, MG, Brazil', 'Copacabana Palace, A Belmond Hotel, RJ, Brazil', 'Sugarloaf Mountain Pão de Açúcar, RJ, Brazil', 'Campos do Jordão SP, Brazil', 'Liberdade São Paulo - SP, Brazil', 'Blumenau, SC, Brazil', 'Balneario Camboriu, SC, Brazil', 'Gramado, RS, Brazil', 'Iguazu Falls, PR, Brazil', 'Bonito, MS, Brazil', 'Porto Velho, RO, Brazil', 'Boa Vista, RR, Brazil', 'Manaus, AM, Brazil', 'Rio Branco, AC, Brazil', 'Cuiaba, MT, Brazil', 'Caldas Novas, GO, Brazil', 'Praça dos Três Poderes, DF, Brazil', 'Palmas, TO, Brazil', 'Fordlândia Aveiro PA, Brazil', 'Belem, PA, Brazil', 'Parque Nacional dos Lençóis Maranhenses, MA, Brazil', 'Jericoacoara, CE, Brazil', 'Fortaleza, CE, Brazil', 'Genipabu Praia de Genipabu, RN, Brazil', 'Cashew of Pirangi, Pirangi do Norte, Parnamirim - RN, Brazil', 'Ponta do Seixas João Pessoa - PB, Brazil', 'Ol